<a href="https://colab.research.google.com/github/DeboraOliver/Random_ML_studies/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Random Forest

# Nova seção

In [ ]:
!pip install -q yfinance

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#if you get an error after executing the code, try adding below. pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader as pdr
import datetime
import yfinance as yf
import requests
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler #normalizar os dados
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

from functools import reduce
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
start = datetime.datetime(2019, 1, 1)
end = datetime.datetime(2022, 10, 27)
# end = datetime.date.today()

SP500 = pdr.DataReader(['sp500'], 'fred', start, end)
SP500 = pd.DataFrame(SP500).reset_index()
SP500.tail()

In [ ]:
vix = pdr.DataReader('VIXCLS', 'fred',start, end).dropna().squeeze()
vix = pd.DataFrame(vix).reset_index()
vix.tail()

In [ ]:
# acoes = ['PETR3.SA','AMER3.SA','GOLL4.SA', 'CVCB3.SA', 'WEGE3.SA', 'MGLU3.SA', 'TOTS3.SA', 'BOVA11.SA']
acoes = ['MGLU3.SA',"^BVSP", "USDBRL=x","CL=F"]

acoes_df = pd.DataFrame()
for acao in acoes:
  #acoes_df[acao] = data.DataReader(acao, data_source='yahoo', start='2015-01-01')['Close']
  acoes_df[acao] = yf.download(acao, start=start)['Close']

acao_bovespa = acoes_df.reset_index().rename(columns={'Date': 'DATE','^BVSP':'bovespa','USDBRL=x':'dolar','CL=F':'bpetroleo'})
acao_bovespa

In [ ]:

df_raw = reduce(lambda x,y: pd.merge(x,y, on='DATE', how='left'), [acao_bovespa, vix, SP500])
df_raw= df_raw.rename(columns={'VIXCLS': 'vix'})
df_raw=df_raw.dropna(axis=0).reset_index().drop('index',axis=1)
# df.isnull().sum()
df_raw.tail()

In [ ]:
figura = plt.figure(figsize=(20,10))
sns.heatmap(df_raw.corr(), annot=True);

In [ ]:
componentes = df_raw.columns[1:].values
caracteristicas = df_raw.loc[:, componentes]
df_normalizando = StandardScaler().fit_transform(caracteristicas)
df_normalizando = pd.DataFrame(data = df_normalizando, columns = componentes)
df_normalizando

In [ ]:
# acoes_df_normalizado = df.copy()
# for i in acoes_df_normalizado.columns[1:]:
#   acoes_df_normalizado[i] = acoes_df_normalizado[i] / acoes_df_normalizado[i][0]

# figura = px.line(title = 'Histórico do preço das ações - normalizado')
# for i in acoes_df_normalizado.columns[1:]:
#   figura.add_scatter(x = acoes_df_normalizado['DATE'], y = acoes_df_normalizado[i], name = i)
# figura.show()

In [ ]:
# # normalizar
# for i in df.columns[1:]:
#   df[i] = acoes_df_normalizado[i] / acoes_df_normalizado[i][0]
# df.tail()

In [ ]:
#detrend
from scipy import signal

detrended = signal.detrend(df.iloc[:,1])

detrended_df = pd.DataFrame(detrended)
detrended_df.plot()


# PCA

In [ ]:
# pca = PCA(n_components= 2)
# principalcomponente = pca.fit_transform(df_normalizando.iloc[:,1:])
# # principalcomponente
# df_pca = pd.DataFrame(data = principalcomponente, columns = ['PC1','PC2'])

# df = pd.concat([df_raw[['DATE']],df_normalizando.iloc[:,0],df_pca],axis = 1)
# df

# T-sne

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components= 3, n_iter = 4000, random_state= 0)
caracteristicas_tsne = tsne.fit_transform(caracteristicas)
# caracteristicas_tsne

#salva os dados sno pandas dataframe
principal_df_tsne = pd.DataFrame(data = caracteristicas_tsne, columns = ['Componente 1','Componente 2','Componente 3'])
#Concatena o dataframe gerado com os componentes princixpais e anexa a vaariável target
final_DF_tsne = pd.concat([principal_df_tsne, df_normalizando[['MGLU3.SA']]], axis = 1)

#Imprime  a "qualidade" do produto gerado. A métrica divergência de KL representa o erro, quanto menor o valor dela, melhor  o modelo.
#Quanto menor for o valor dessa métrica, melhor é o modelo
print('Kullback-Leibler divergence t-SNE: ', tsne.kl_divergence_)

In [ ]:
figura = px.line(title = 'Histórico do preço das ações - normalizado')
for i in df.columns[1:]:
  figura.add_scatter(x = df['DATE'], y = df[i], name = i)
figura.show()

In [ ]:
#o que eu quero prever: o valor da ação
y_objetivo = df.iloc[len(df) - 60: ,1].values
#previsor
X_previsor = df.iloc[len(df) - 60:,3:].values
# X_previsor

In [ ]:
X_previsor_treinamento, X_previsor_teste, y_objetivo_treinamento, y_objetivo_teste = train_test_split(X_previsor, y_objetivo, test_size = 0.3, random_state = 0)
#deixamos 70% para treinamento e 30% para testes
#random state = 0 para que sempre seja a msm divisão com os msm registros
X_previsor_treinamento.shape, y_objetivo_treinamento.shape

In [ ]:
# regressor_random_forest_saude = RandomForestRegressor(n_estimators = 10)
regressor_random_forest_acoes = RandomForestRegressor(n_estimators=1000)
regressor_random_forest_acoes.fit(X_previsor_treinamento, y_objetivo_treinamento)

print("Score de treinamento ", regressor_random_forest_acoes.score(X_previsor_treinamento, y_objetivo_treinamento))
print("Score de teste ", regressor_random_forest_acoes.score(X_previsor_teste, y_objetivo_teste))

In [ ]:
previsoes = regressor_random_forest_acoes.predict(X_previsor_teste)
previsoes[0:5]

In [ ]:
y_objetivo_teste[0:5]

In [ ]:
figura = plt.figure(figsize=(20,10))
plt.plot(y_objetivo_teste, label='Expected')
plt.plot(previsoes, label='Predicted')
plt.legend()
plt.show()

In [ ]:

mean_absolute_error(y_objetivo_teste, previsoes)

In [ ]:
# Calculate feature importance and store in pandas series
feature_imp = pd.Series(regressor_random_forest_acoes.feature_importances_, index=df.iloc[:,2:].columns).sort_values(ascending=False)
feature_imp

In [ ]:
last_60_days = df.iloc[:,2:7].values
last_60_days
# scaler = MinMaxScaler(feature_range=(0,1))
# X_test = []
# #append the past 60 days
# X_test.append(last_60_days)
# #convert the x_test data set to a numpy array
# X_test = np.array(X_test)
# #Reshape the data
# X_test = np.reshape(X_test.shape[0], X_test.shape[1])
#Get the predict scaled price
# pred_price = regressor_random_forest_acoes.predict(last_60_days)
#undo the scaling
# pred_price = StandardScaler.inverse_transform(pred_price)
# print(pred_price[-5:])

# last_60_days = new_df[-60:].values
# #Scale the data to be values between 0 and 1
# last_60_days_scaled = scaler.transform(last_60_days)
# #Create an empty list
# X_test = []
# # #append the past 60 days
# X_test.append(last_60_days_scaled)
# # #convert the x_test data set to a numpy array
# X_test = np.array(X_test)
# # #Reshape the data
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
# #Get the predict scaled price
# pred_price = model.predict(X_test)
# #undo the scaling
# pred_price = scaler.inverse_transform(pred_price)
# print(pred_price)

In [ ]:
df.tail()

In [ ]:
last_60_days.shape